In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("b_depressed.csv")
df.head()

,Survey_id,Ville_id,sex,Age,Married,Number_children,education_level,total_members,gained_asset,durable_asset,...,incoming_salary,incoming_own_farm,incoming_business,incoming_no_business,incoming_agricultural,farm_expenses,labor_primary,lasting_investment,no_lasting_investmen,depressed
0,926,91,1,28,1,4,10,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,0
1,747,57,1,23,1,3,8,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,1
2,1190,115,1,22,1,3,9,5,28912201,22861940,...,0,0,0,0,30028818,31363432,0,28411718,28292707.0,0
3,1065,97,1,27,1,2,10,4,52667108,19698904,...,0,1,0,1,22288055,18751329,0,7781123,69219765.0,0
4,806,42,0,59,0,4,10,6,82606287,17352654,...,1,0,0,0,53384566,20731006,1,20100562,43419447.0,0


In [3]:
df.drop(['Survey_id','Ville_id'], axis = 1, inplace = True)

In [4]:
df.isnull().sum()

sex                       0
Age                       0
Married                   0
Number_children           0
education_level           0
total_members             0
gained_asset              0
durable_asset             0
save_asset                0
living_expenses           0
other_expenses            0
incoming_salary           0
incoming_own_farm         0
incoming_business         0
incoming_no_business      0
incoming_agricultural     0
farm_expenses             0
labor_primary             0
lasting_investment        0
no_lasting_investmen     20
depressed                 0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.isnull().sum()

sex                      0
Age                      0
Married                  0
Number_children          0
education_level          0
total_members            0
gained_asset             0
durable_asset            0
save_asset               0
living_expenses          0
other_expenses           0
incoming_salary          0
incoming_own_farm        0
incoming_business        0
incoming_no_business     0
incoming_agricultural    0
farm_expenses            0
labor_primary            0
lasting_investment       0
no_lasting_investmen     0
depressed                0
dtype: int64

In [7]:
print('\nEl numero de filas y columnas en el data set: ',df.shape)
print('')


El numero de filas y columnas en el data set:  (1409, 21)



In [8]:
# Genero datos de entrenamiento y test
X = df.drop('depressed',axis=1) # Variable independiente
y = df['depressed'] # Variable dependiente

# KNN 
## RFE

In [9]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [10]:
model = LogisticRegression(solver='lbfgs', max_iter=4000)

In [11]:
# Se aplica RFE, Se pide los 5 atributos top
rfe = RFE(estimator=model, n_features_to_select=5)
rfe = rfe.fit(X,y)

In [12]:
# Imprimo la seleccion de atributos
print(rfe.support_)
print(rfe.ranking_)
print(X.columns.values)

[False False False False False False  True False  True  True False False
 False False False  True  True False False False]
[10 11  9  8  6  7  1  5  1  1  3 16 15 13 12  1  1 14  2  4]
['sex' 'Age' 'Married' 'Number_children' 'education_level' 'total_members'
 'gained_asset' 'durable_asset' 'save_asset' 'living_expenses'
 'other_expenses' 'incoming_salary' 'incoming_own_farm'
 'incoming_business' 'incoming_no_business' 'incoming_agricultural'
 'farm_expenses' 'labor_primary' 'lasting_investment'
 'no_lasting_investmen']


In [13]:
# Los 5 atributos top son 'education.num' ,  'marital.status' , 'sex_ Male'
X_rfe = X[['gained_asset','save_asset','living_expenses','incoming_agricultural', 'farm_expenses']]

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_rfe,y,test_size=0.2,random_state=23)

In [15]:
print("Numero de datos de entrenamiento: ", len(X_train))
print("Numero de datos en test: ", len(X_test))

Numero de datos de entrenamiento:  1127
Numero de datos en test:  282


## Aplicando KNN

In [16]:
leaf_size = list(range(1,20))
n_neighbors = list(range(1,10))
p=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [18]:
from sklearn.model_selection import GridSearchCV
gsv = GridSearchCV(knn, hyperparameters, cv=10)

In [19]:
best_model = gsv.fit(X_train, y_train)

In [20]:
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Best leaf_size: 1
Best p: 2
Best n_neighbors: 9


In [21]:
best_knn = KNeighborsClassifier(leaf_size=1, n_neighbors=9, p=2)
best_knn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=1, n_neighbors=9)

In [22]:
# Realizamos la prediccion
y_pred = best_knn.predict(X_test)

In [23]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

acc_knn = metrics.accuracy_score(y_test,y_pred)
acc_knn #0.8439716312056738

0.8439716312056738